In [6]:
%matplotlib inline

import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np

import json
import os
import glob
import datetime

# Essay scoring

## Development set

In [16]:
# Gather all params.
results = []

for d in glob.glob("../output/*"):
    if not os.path.exists(os.path.join(d, "param.txt")):
        continue
    
    prm = dict([p.strip().split("=", 1) for p in open(os.path.join(d, "param.txt"))])
    prm["Directory"] = os.path.basename(d)
    prm["Time"] = str(datetime.datetime.fromtimestamp(os.stat(d).st_mtime))[:19]
    prm["Fold"] = [i for i in range(5) if os.path.exists(os.path.join(d, "regression_f{}.hdf5".format(i)))]
    prm["Eval"] = [i for i in range(5) if os.path.exists(os.path.join(d, "prediction_f{}.json".format(i)))]

    mse_pool = []
    
    for i in range(5):
        pjs = os.path.join(d, "prediction_f{}.json".format(i))
        
        if os.path.exists(pjs):
            pred = json.load(open(pjs))
            mse_pool += [pred["MSE"]]

    mse_pool = np.array(mse_pool)
    prm["MSEstd"] = mse_pool.std()    
    prm["MSE"] = mse_pool.mean()
    prm["MSEs"] = mse_pool
    
#     l = pickle.load(open("{}/logs.pickle".format(d), "rb"))
#     df = pd.DataFrame(l["logs_e"])
#     prm["R_loss"] = df["loss"].min()
#     prm["R_val_loss"] = df["val_loss"].min()
    
    results += [prm]

cols = "Time Fold Eval MSE MSEstd MSEs di_aware dropout emb_fix enc_fix pseq preenc Directory".split()

pd.set_option("display.max_colwidth", 200)
df = pd.DataFrame(results)
df = df[df["di_aware"] == "False"]
df[cols].sort_values(by="Time", ascending=False)

/home/naoya-i/py36/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: Mean of empty slice.


,Time,Fold,Eval,MSE,MSEstd,MSEs,di_aware,dropout,emb_fix,enc_fix,pseq,preenc,Directory
8,2019-01-08 19:21:42,"[0, 1, 2, 3, 4]","[0, 1, 2, 3, 4]",0.189276,0.021962,"[0.1722483500131174, 0.1794667317586682, 0.16374400186657234, 0.2143731473086255, 0.21654933758099426]",False,0.7,False,False,True,output_enc/750570aed2d16633ecbe4237d2d95b71,6f27e738794f9eb73bd73c0c868d4cb2
0,2019-01-08 19:20:50,"[0, 1, 2, 3, 4]","[0, 1, 2, 3, 4]",0.195326,0.020017,"[0.17976007916678496, 0.18664661003761268, 0.1720895049572962, 0.2160131768946912, 0.22211994933249912]",False,0.7,False,False,True,output_enc/a87b827fa7c5151192542ecb2c3af4d2,f925ce09bd56d0905e11730817dc4439
1,2019-01-08 19:16:04,"[0, 1, 2, 3]",[],NaN,NaN,[],False,0.7,True,True,True,output_enc/a87b827fa7c5151192542ecb2c3af4d2,3455bddfc5e1a6f41d846b8d6a3e092f
9,2019-01-08 18:24:11,"[0, 1, 2, 3, 4]","[0, 1, 2, 3, 4]",0.188923,0.023066,"[0.1732169700966405, 0.17464527016437387, 0.16342075147891053, 0.21352521893260346, 0.21980865044014983]",False,0.7,True,True,True,output_enc/750570aed2d16633ecbe4237d2d95b71,fa37bf66f2563eec16382c1eac16a108
5,2019-01-08 17:32:24,[],[],NaN,NaN,[],False,0.7,True,True,True,output_enc/750570aed2d16633ecbe4237d2d95b71,d66f0a27964dd57535957583bec9c684
7,2019-01-08 17:08:49,[],[],NaN,NaN,[],False,0.7,False,False,True,output_enc/750570aed2d16633ecbe4237d2d95b71,2674fd3a66a463d7fd62804995734663
3,2019-01-08 17:06:19,[],[],NaN,NaN,[],False,0.7,False,False,True,output_enc/a87b827fa7c5151192542ecb2c3af4d2,e0ab2e7b43b37719666bb6e1b0e58e10
6,2018-12-28 22:27:22,[1],[1],0.273942,0.000000,[0.27394229360393907],False,0.5,False,False,True,None,6c41a1e4fe316bada6d0b2b14433374b
2,2018-12-28 22:22:51,[1],[1],0.299845,0.000000,[0.2998450864722913],False,0.5,False,False,True,None,1f17c78951114d7214f77372b62f4d0a
4,2018-12-28 22:19:10,[1],[1],0.317584,0.000000,[0.31758360822801335],False,0.5,True,False,False,None,4b51b59d853fff7dca91b9f13b017f86


## Test set

In [26]:
# Gather all results.
results = []

for d in glob.glob("../output/*"):
    if not os.path.exists(os.path.join(d, "param.txt")):
        continue
        
    prm = dict([p.strip().split("=", 1) for p in open(os.path.join(d, "param.txt"))])
    
    p = [json.load(open(os.path.join(d, "prediction_f{}.json".format(i)))) for i in range(5) if os.path.exists(os.path.join(d, "prediction_f{}.json".format(i)))]
    p = np.array([x["MSE"] for x in p])
    
    prm["Directory"] = os.path.basename(d)
    prm["MSEs"] = p
    prm["MSE"] = p.mean(), p.std()
    
    results += [prm]
        
df = pd.DataFrame(results)
df[["Directory", "MSEs", "MSE", "preenc", "enc_fix", "emb_fix", "pretrained", "pseq"]]

/home/naoya-i/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice.
  from ipykernel import kernelapp as app


,Directory,MSEs,MSE,preenc,enc_fix,emb_fix,pretrained,pseq
0,f925ce09bd56d0905e11730817dc4439,"[0.17976007916678496, 0.18664661003761268, 0.1720895049572962, 0.2160131768946912, 0.22211994933249912]","(0.19532586407777686, 0.020017189672814253)",output_enc/a87b827fa7c5151192542ecb2c3af4d2,False,False,False,True
1,0ade997a0a7df4d5029eba715a4f1ca9,[],"(nan, nan)",output_enc/9780456c95e7c048e2501106fd40c716,False,False,False,True
2,b65364664e64110a43ca71623a066182,"[0.42131071507617945, 0.32779798986995845, 0.38009449324749595, 0.3451958241970267, 0.35285052162591823]","(0.3654499088033158, 0.032623712329671886)",output_enc/9780456c95e7c048e2501106fd40c716,False,False,False,False
3,46f970b1c6379f85b7ccc1fe68a8af14,"[0.17270962312103763, 0.2207709531401928, 0.18877799411561982, 0.21240548011506646, 0.22274311408665057]","(0.20348143291571344, 0.019558496261868393)",output_enc/9780456c95e7c048e2501106fd40c716,True,True,False,True
4,3455bddfc5e1a6f41d846b8d6a3e092f,"[0.17598452566143735, 0.17643581246378712, 0.15157045217080076, 0.2074851881946586, 0.2187000135078771]","(0.18603519839971222, 0.024116696182171826)",output_enc/a87b827fa7c5151192542ecb2c3af4d2,True,True,False,True
5,0aa2570ced889c4e88ae1554253cb412,"[0.18851345297596922, 0.16698824099978538, 0.1784539630179947, 0.22683654581530616, 0.2222658611700897]","(0.19661161279582903, 0.023851673443049014)",output_enc/9780456c95e7c048e2501106fd40c716,False,False,False,True
6,1f17c78951114d7214f77372b62f4d0a,[0.2998450864722913],"(0.2998450864722913, 0.0)",None,False,False,False,True
7,e0ab2e7b43b37719666bb6e1b0e58e10,[],"(nan, nan)",output_enc/a87b827fa7c5151192542ecb2c3af4d2,False,False,False,True
8,4b51b59d853fff7dca91b9f13b017f86,[0.31758360822801335],"(0.31758360822801335, 0.0)",None,False,True,True,False
9,d66f0a27964dd57535957583bec9c684,[],"(nan, nan)",output_enc/750570aed2d16633ecbe4237d2d95b71,True,True,False,True


In [ ]:
tn16pn10_pre_sentshuf_finetune = "0cb7a2429b6cc73297413c20570c824f"
tn16pn10_pre_sentshuf_nofinetune = "752aa6160e706a6ffe0f91a1e423b40a"

# Encoder pretraining

In [26]:
# Gather all params.
results = []

for d in glob.glob("../output_enc/*"):
    if not os.path.exists(os.path.join(d, "param.txt")):
        continue
    
    prm = dict([p.strip().split("=", 1) for p in open(os.path.join(d, "param.txt"))])
    prm["Directory"] = os.path.basename(d)
    prm["Time"] = str(datetime.datetime.fromtimestamp(os.stat(d).st_mtime))[:19]
    
    l = pickle.load(open("{}/logs.pickle".format(d), "rb"))
    df = pd.DataFrame(l["logs_e"])
    prm["R_acc"] = df["acc"].max()
    prm["R_val_acc"] = df["val_acc"].max()
    
    results += [prm]

cols = "Time R_acc R_val_acc dropout emb_fix enc_fix shuf Directory".split()

pd.set_option("display.max_colwidth", 50)
df = pd.DataFrame(results)
df[cols].sort_values(by="Time", ascending=False)

,Time,R_acc,R_val_acc,dropout,emb_fix,enc_fix,shuf,Directory
3,2018-12-27 14:18:45,0.993037,0.903384,0.7,False,False,di,a87b827fa7c5151192542ecb2c3af4d2
5,2018-12-27 13:25:00,0.986483,0.825873,0.5,False,False,di,6abbf82fd461ebcd2ac61867427b2a1e
1,2018-12-27 13:02:47,0.996313,0.853712,0.3,False,False,di,da6836f961365a7b348281a00e48bc34
2,2018-12-27 11:37:36,0.992900,0.936681,NaN,NaN,NaN,NaN,clipnorm=5.0_dropout=0.7_emb_dim=50_emb_fix=Fa...
4,2018-12-27 11:32:34,0.902512,0.743996,NaN,NaN,NaN,NaN,clipnorm=5.0_dropout=0.7_emb_dim=50_emb_fix=Fa...
0,2018-12-27 11:32:27,0.925587,0.742904,NaN,NaN,NaN,NaN,clipnorm=5.0_dropout=0.5_emb_dim=50_emb_fix=Fa...


In [2]:
### Score_normalized
# TN16
CUDA_VISIBLE_DEVICES=0 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-LSTMdim 300 \
    --gradientclipnorm 5 --meanovertime \
    --pre-trained --fix-embedding

fold_0: MSE: 0.3643280911204178, MAE: 0.4622013795375824
fold_1: MSE: 0.3197095480935262, MAE: 0.39644437405600474
fold_2: MSE: 0.4364101866079787, MAE: 0.438516518369836   
fold_3: MSE: 0.32989512169457946, MAE: 0.4378773703503965
fold_4: MSE: 0.3643280911204178, MAE: 0.4622013795375824
            
MSE: 0.362

SyntaxError: invalid syntax (<ipython-input-2-e702d4088bd6>, line 3)

In [ ]:
fold0: MSE: 0.19625852776829697 MAE: 0.3492366951704025
fold1: 

In [5]:
# Gather all results.
results = []

for d in glob.glob("../output/*"):
    if not os.path.exists(os.path.join(d, "prediction_f1.json")):
        continue
        
    prm = dict([p.strip().split("=", 1) for p in open(os.path.join(d, "param.txt"))])
    
    p = json.load(open(os.path.join(d, "prediction_f1.json")))
    prm["MSE"] = p["MSE"]
    
    results += [prm]
        
df = pd.DataFrame(results)
df[["MSE", "preenc", "enc_fix", "emb_fix", "pretrained", "pseq"]]

,MSE,preenc,enc_fix,emb_fix,pretrained,pseq
0,0.335248,None,False,True,True,False
1,0.319710,None,False,True,True,False


# Command repo

In [1]:
# Command for training:

# TN16
CUDA_VISIBLE_DEVICES=0 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-grudim 300 \
    --gradientclipnorm 5 --meanovertime \
    --pre-trained --fix-embedding

# TN16+PN10
CUDA_VISIBLE_DEVICES=0 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --pre-trained --fix-embedding \
    --persing-seq --pseq-embedding-dim 16 --pseq-encoder-dim 64

# TN16+PN10+pretrain(di. shuffle, fixed)
CUDA_VISIBLE_DEVICES=0 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --persing-seq --pseq-embedding-dim 16 --pseq-encoder-dim 64 \
    --fix-encoder --fix-embedding \
    --pretrained-encoder output_enc/a87b827fa7c5151192542ecb2c3af4d2

# TN16+PN10+pretrain(di. shuffle, not fixed)
CUDA_VISIBLE_DEVICES=0 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --persing-seq --pseq-embedding-dim 16 --pseq-encoder-dim 64 \
    --pretrained-encoder output_enc/a87b827fa7c5151192542ecb2c3af4d2

# TN16+PN10+pretrain(sent. shuffle, fixed)
CUDA_VISIBLE_DEVICES=1 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --persing-seq --pseq-embedding-dim 16 --pseq-encoder-dim 64 \
    --fix-encoder --fix-embedding \
    --pretrained-encoder output_enc/clipnorm=5.0_dropout=0.7_emb_dim=50_emb_fix=False_enc_fix=False_model_type=nea_mot=True_pretrained=False_shuf=sentence

# TN16+PN10+pretrain(sent. shuffle, not fixed)
CUDA_VISIBLE_DEVICES=1 python src/train.py \
    --fold 1 \
    --model-type nea --dropout 0.5 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --persing-seq --pseq-embedding-dim 16 --pseq-encoder-dim 64 \
    --pretrained-encoder output_enc/clipnorm=5.0_dropout=0.7_emb_dim=50_emb_fix=False_enc_fix=False_model_type=nea_mot=True_pretrained=False_shuf=sentence

SyntaxError: invalid syntax (<ipython-input-1-dba85e4206fa>, line 4)

In [ ]:
# Command for sentence encoder pretraining:
CUDA_VISIBLE_DEVICES=1 python src/train_enc.py \
    --model-type nea --dropout 0.3 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --shuffle-type di

CUDA_VISIBLE_DEVICES=1 python src/train_enc.py \
    --model-type nea --dropout 0.3 \
    --embedding-dim 50 --aggregation-grudim 100 \
    --gradientclipnorm 5 --meanovertime \
    --shuffle-type sentence

In [ ]:
# Command for evaluation
CUDA_VISIBLE_DEVICES=1 python src/eval.py \
    --fold 1 \
    --model-dir output/cbc428f99e04b33c2dcb221e7331e07d

In [11]:
import pickle

logs = pickle.load(open("../output/2cfef26bfd48471a2c54726988f72420/logs_f1.pickle", "rb"))
df = pd.DataFrame(logs["logs_e"])
df

,loss,mean_squared_error,val_loss,val_mean_squared_error
0,0.057288,0.057288,0.028863,0.028863
1,0.043255,0.043255,0.035102,0.035102
2,0.041587,0.041587,0.033113,0.033113
3,0.040196,0.040196,0.034093,0.034093
4,0.039479,0.039479,0.027005,0.027005
5,0.038605,0.038605,0.026719,0.026719
6,0.039017,0.039017,0.027171,0.027171
7,0.038305,0.038305,0.026136,0.026136
8,0.037606,0.037606,0.026472,0.026472
9,0.036938,0.036938,0.031719,0.031719


In [12]:
df.describe()

,loss,mean_squared_error,val_loss,val_mean_squared_error
count,35.000000,35.000000,35.000000,35.000000
mean,0.030731,0.030731,0.028449,0.028449
std,0.009068,0.009068,0.002609,0.002609
min,0.016624,0.016624,0.025675,0.025675
25%,0.023180,0.023180,0.026428,0.026428
50%,0.029469,0.029469,0.028146,0.028146
75%,0.037357,0.037357,0.029157,0.029157
max,0.057288,0.057288,0.035102,0.035102
